In [4]:
from pydantic import BaseModel, Field
from typing import List

class Slide(BaseModel):
    title: str = Field(
        description="title for this slide.",
    )
    text: str = Field(
        description="text for this slide.",
    )

class Slides(BaseModel):
    slides: List[Slide] = Field(
        description="Slides to present the document in a presentation.",
    )

In [1]:
with open("texts/sample.txt", "r") as f:
    doc =f.read()

In [14]:
from google import genai
from pydantic import BaseModel, Field
from typing import List, Optional

client = genai.Client()

prompt = """
make slides for the following text.
{text}
"""

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=prompt.format(text=doc),
    config={
        "response_mime_type": "application/json",
        "response_json_schema": Slides.model_json_schema(),
    },
)

res = Slides.model_validate_json(response.text)
print(res)

slides=[Slide(title='OpenAI: From Idealistic Non-Profit to Global AI Leader', text="An overview of OpenAI's transformation since its 2015 launch, covering its original mission, key milestones, and the evolving debate over AI safety and openness."), Slide(title='The Founding Vision (December 2015)', text='Launched as a non-profit research lab unconstrained by financial returns. Mission: Advance digital intelligence to benefit humanity. Key features: $1B in funding, top-tier team (Sutskever, Karpathy, etc.), and a commitment to openness and safety.'), Slide(title='Early Community Reactions', text="Initial themes included excitement over the star-studded team and debates on whether deep learning would plateau. Discussions touched on AI safety vs. 'fear-mongering,' the ethics of AI, and future societal impacts like job loss and propaganda."), Slide(title='2016–2018: The Open Research Phase', text='Focused on transparent research and community tools. Key releases included OpenAI Gym, Univer

In [16]:
len(res.slides)

11

In [2]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]


In [7]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "presentation_slides",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "slides": {
                    "type": "array",
                    "description": "Slides to present the document in a presentation.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "title": {
                                "type": "string",
                                "description": "title for this slide."
                            },
                            "text": {
                                "type": "string",
                                "description": "text for this slide."
                            }
                        },
                        "required": ["title", "text"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["slides"],
            "additionalProperties": False
        }
    }
}

In [5]:
def generate_structured_output(prompt: str, model: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "response_format": response_format
    }
    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    return result

In [8]:
prompt = """
make slides for the following text.
{text}
"""

In [9]:
res = generate_structured_output(prompt.format(text=doc), "google/gemini-3-flash-preview")

In [13]:
slides = res["choices"][0]["message"]["content"]

In [21]:
import json

In [23]:
json_data =json.loads(slides)

In [26]:
len(json_data['slides'])


8

In [27]:
json_data['slides'][0]

{'title': 'OpenAI 2015: The Original Vision',
 'text': 'OpenAI launched in December 2015 as a non-profit AI research lab with $1B in funding. Its mission was to advance digital intelligence to benefit humanity, unconstrained by financial return. Key early focuses included openness (shared code and patents), safety as an extension of human will, and deep learning excellence led by a star founding team.'}